In [1]:
import os
import re
from pathlib import Path
from shutil import copyfile
import pandas as pd

In [2]:
data_folder = './csv_data'
total_samples = 2400

In [3]:
df_count = pd.DataFrame(columns=['topic', 'count', 'proportion_count'])

csv_dfs = {}
total_count = 0

entries = os.listdir(data_folder)
regex = re.compile('data_([a-zA-Z]*)_nodups_wobj\.csv')

for filename in entries:
    if filename.startswith('.'): continue
    matched = regex.search(filename)
    if matched:
        topic_name = matched.group(1)
        csv_path = os.path.join(data_folder, filename)
        csv_dfs[topic_name] = pd.read_csv(csv_path, index_col=0)
        row_count = len(csv_dfs[topic_name].index)
        total_count += row_count
        df_count.loc[len(df_count)] = [topic_name, row_count, -1]
        print('{:<13}{:<7}'.format(topic_name, row_count))


Immigration  6854   
AsianHate    1602   
Boomer       2184   
Vaccine      2671   
Mask         20149  


In [4]:
df_count['proportion_count'] = df_count['count'].apply(lambda x: round((x/total_count)*total_samples))

In [5]:
df_count.to_csv('./csv_data/sample_count.csv', index=False)
df_count

,topic,count,proportion_count
0,Immigration,6854,492
1,AsianHate,1602,115
2,Boomer,2184,157
3,Vaccine,2671,192
4,Mask,20149,1445


In [6]:
df_count['proportion_count'].sum()

2401

In [7]:
df_samples = pd.DataFrame(columns=['topic', 'hashtag', 'tweet_id', 'image_path', 'sample_path', 'body_text', 'image_text'])

for _, row in df_count.iterrows():
    topic_name = row['topic']
    df_topic = csv_dfs[topic_name].sample(n=row['proportion_count'], random_state=42)
    df_topic.insert(loc=0, column='topic', value=topic_name)
    df_topic.insert(loc=4, column='sample_path', value=-1)
    df_topic.insert(loc=len(df_topic.columns), column='image_text', value=-1)
    df_samples = df_samples.append(df_topic)

df_samples.reset_index(drop=True, inplace=True)
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text
0,Immigration,immigration,1298354080579035137,./data_Immigration/immigration/tweets4/EgSuFY5...,-1,"@LeftIsBest10 nah, cos they are both using the...",-1
1,Immigration,cdnpoli,1291074721195397120,./data_Immigration/cdnpoli/tweets1/EerRnVOXgAA...,-1,@SethDKlein This is my underground version ver...,-1
2,Immigration,cdnpoli,1297904197569056768,./data_Immigration/cdnpoli/tweets4/EgMU7eQWoAI...,-1,"Join Us on Sunday, August 30th, 6:30pm!\nVirtu...",-1
3,Immigration,refugees,1296060608635379712,./data_Immigration/refugees/tweets3/EfyIQTXXsA...,-1,#CleanCookingIs more than preparing meals...\n...,-1
4,Immigration,cdnpoli,1298211948895571968,./data_Immigration/cdnpoli/tweets4/EgQs4zzXgAA...,-1,Sounds like the Canadian version of @realDonal...,-1
...,...,...,...,...,...,...,...
2396,Mask,facemask,1293301322524037120,./data_Mask/facemask/tweets2/EfK6stXX0AUMHN4.jpg,-1,Excited to share the latest addition to my #et...,-1
2397,Mask,NoMasks,1293053718292766720,./data_Mask/NoMasks/tweets2/EfHZf-4XkAEkRSF.jpg,-1,@ShotgunHolliday 🤣 Nice! \n#CovidHoax #NoMasks...,-1
2398,Mask,facemask,1292890249446797317,./data_Mask/facemask/tweets2/EfFE1IsWAAIntrB.jpg,-1,👉 https://t.co/JrYD3TuXl7\n\nThese face masks ...,-1
2399,Mask,WearAMask,1294605153031589888,./data_Mask/WearAMask/tweets3/EfdchVCWkAEjgqG.jpg,-1,#WearAMask 😷 to protect yourself and others . ...,-1


## Copy selected samples and generate sample path (Please run cells below)

In [1]:
import os
from pathlib import Path
from shutil import copyfile
import pandas as pd

In [2]:
df_count = pd.read_csv('./csv_data/sample_count.csv')
df_samples = pd.read_csv('./csv_data/sample_data.csv')

In [3]:
def get_sample_path_and_copy(df, topic, annot_data_path):
    # if the sample_path already exists
    if df['sample_path'] != -1:
        return df['sample_path']

    # if topic is not the selected topic
    if df['sample_path'] == -1 and df['topic'] != topic:
        return -1
    
    image_name = df['image_path'].split('/')[-1]
    dst_path = os.path.join(annot_data_path, image_name)
    copyfile(df['image_path'], dst_path)
    return dst_path

In [4]:
sample_folder = './annotation_data'
Path(sample_folder).mkdir(exist_ok=True)

for _, row in df_count.iterrows():
    topic_name = row['topic']
    topic_data_path = os.path.join('./', 'data_' + topic_name)
    annot_data_path = os.path.join(sample_folder, topic_name)
    # if data_{topic_name} folder doesn't exist
    if not Path(topic_data_path).is_dir():
        print('{} not exists; pass this topic'.format(topic_data_path))
        continue

    # if the sampled data for this topic already exists
    if Path(annot_data_path).is_dir():
        print('{} already exist; pass this topic'.format(annot_data_path))
        continue

    Path(annot_data_path).mkdir()

    df_samples['sample_path'] = df_samples.apply(get_sample_path_and_copy, axis=1, topic=topic_name, annot_data_path=annot_data_path)

    print('Topic \"{}\" processed'.format(topic_name))

Topic "Immigration" processed
./data_AsianHate not exists; pass this topic
./data_Boomer not exists; pass this topic
Topic "Vaccine" processed
./data_Mask not exists; pass this topic


In [5]:
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text
0,Immigration,immigration,1298354080579035137,./data_Immigration/immigration/tweets4/EgSuFY5...,./annotation_data/Immigration/EgSuFY5WsAEjjyv.png,"@LeftIsBest10 nah, cos they are both using the...",-1
1,Immigration,cdnpoli,1291074721195397120,./data_Immigration/cdnpoli/tweets1/EerRnVOXgAA...,./annotation_data/Immigration/EerRnVOXgAACHWo.jpg,@SethDKlein This is my underground version ver...,-1
2,Immigration,cdnpoli,1297904197569056768,./data_Immigration/cdnpoli/tweets4/EgMU7eQWoAI...,./annotation_data/Immigration/EgMU7eQWoAIY5lY.jpg,"Join Us on Sunday, August 30th, 6:30pm!\nVirtu...",-1
3,Immigration,refugees,1296060608635379712,./data_Immigration/refugees/tweets3/EfyIQTXXsA...,./annotation_data/Immigration/EfyIQTXXsAAIRXH.jpg,#CleanCookingIs more than preparing meals...\n...,-1
4,Immigration,cdnpoli,1298211948895571968,./data_Immigration/cdnpoli/tweets4/EgQs4zzXgAA...,./annotation_data/Immigration/EgQs4zzXgAAsL7s.jpg,Sounds like the Canadian version of @realDonal...,-1
...,...,...,...,...,...,...,...
2396,Mask,facemask,1293301322524037120,./data_Mask/facemask/tweets2/EfK6stXX0AUMHN4.jpg,-1,Excited to share the latest addition to my #et...,-1
2397,Mask,NoMasks,1293053718292766720,./data_Mask/NoMasks/tweets2/EfHZf-4XkAEkRSF.jpg,-1,@ShotgunHolliday 🤣 Nice! \n#CovidHoax #NoMasks...,-1
2398,Mask,facemask,1292890249446797317,./data_Mask/facemask/tweets2/EfFE1IsWAAIntrB.jpg,-1,👉 https://t.co/JrYD3TuXl7\n\nThese face masks ...,-1
2399,Mask,WearAMask,1294605153031589888,./data_Mask/WearAMask/tweets3/EfdchVCWkAEjgqG.jpg,-1,#WearAMask 😷 to protect yourself and others . ...,-1
